In [1]:
import os
import shutil
import yaml
import re

In [2]:
arxiv_number_list = [
    "1010.1819",
    "1106.6060",
    "1208.0116",
    "1310.2674",
    "1812.04213",
    "2004.04168",
    "2008.08998",
    "2012.04554",
    "2108.02159",
    "2110.11330",
    "2111.01152",
    "2112.07523",
    "2308.03843",
    "2308.07488",
]

In [7]:
with open('../Naming.yaml', 'r') as f:
    naming = yaml.safe_load(f)
def replace_title(text):
    pattern = r'^##\s*(.*)$'
    # while 1:
    matches = re.finditer(pattern, text, re.MULTILINE)
    for match in matches:
        sec_title=match.group(0)
        title=sec_title[3:].strip()
        text=text.replace(sec_title, f"# {naming[title]}")
    return text
def replace_separator(text,):
    return text.replace('===','')
def trim_spaces_in_latex(text):
    '''Trim spaces in latex, e.g. $ 1 $ -> $1$, which otherwise cannot be recognized by markdown in latex'''
    pattern = r'(?<!\$)\$(\s*)(.*?)(\s*)\$(?!\$)'
    def replace(match):
        return f'${match.group(2)}$'
    return re.sub(pattern, replace, text)
def preprocessing(text):
    text=replace_title(text)
    text=replace_separator(text)
    text=trim_spaces_in_latex(text)

    return text
def postprocessing(text):
    text=text.replace(r'\bf',r'\bm')
    return text

In [4]:
# Create folders, move from ../arxiv_number/arxiv_number_auto.md to ./arxiv_number/arxiv_number_execution.md
# Then do some preprocessing

for arxiv_number in arxiv_number_list:
    # Check if folder exists
    folder_name = arxiv_number
    if not os.path.exists(folder_name):
        # Create folder
        os.makedirs(folder_name, exist_ok=True)
    
    source_file=os.path.join('..',arxiv_number,f'{arxiv_number}_auto.md')
    target_file=os.path.join(folder_name,f'{folder_name}_execution.md')
    shutil.copyfile(source_file,target_file)

    with open(os.path.join(arxiv_number, f"{arxiv_number}_execution.md"), "r") as tex_file:
        execution = tex_file.read()
    execution_1=preprocessing(execution)
    with open(os.path.join(arxiv_number, f"{arxiv_number}_execution.md"), "w") as tex_file:
        tex_file.write(execution_1)
    


In [8]:
# Do some postprocessing, replacing macros etc

for arxiv_number in arxiv_number_list:
    with open(os.path.join(arxiv_number, f"{arxiv_number}_execution.md"), "r") as tex_file:
        execution = tex_file.read()
    execution_1=postprocessing(execution)
    with open(os.path.join(arxiv_number, f"{arxiv_number}_execution.md"), "w") as tex_file:
        tex_file.write(execution_1)

In [14]:
# Generate latex file for each arxiv number
with open('execution.tex','r') as tex_file:
    execution_tex=tex_file.read()

for arxiv_number in arxiv_number_list:
    with open(os.path.join(arxiv_number, f"{arxiv_number}_execution.tex"), "w") as tex_file:
        tex_file.write(execution_tex.replace('arxiv',arxiv_number))

In [3]:
arxiv_number_list[11]

'2112.07523'

In [4]:
# Compile all the latex using 'lualatex {arxiv_number}_execution.tex'
for arxiv_number in arxiv_number_list[11:]:
    os.system(f"cd {arxiv_number} && lualatex {arxiv_number}_execution.tex && cd ..")
    print(arxiv_number)


This is LuaHBTeX, Version 1.17.0 (TeX Live 2023) 
 restricted system commands enabled.
(./2112.07523_execution.tex
LaTeX2e <2023-06-01> patch level 1
L3 programming layer <2023-08-29>
 (/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/local/texlive/2023/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/amsmath/amsgen.sty))
(/usr/local/texlive/2023/texmf-dist/tex/latex/amsmath/amsbsy.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/amsmath/amsopn.sty))
(/usr/local/texlive/2023/texmf-dist/tex/latex/breqn/breqn.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/l3kernel/expl3.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/l3backend/l3backend-luatex.def))
(/usr/local/texlive/

In [11]:
# check integrity, print missing pdf
for arxiv_number in arxiv_number_list:
    file_path = f"{arxiv_number}/{arxiv_number}_execution.pdf"
    if not os.path.exists(file_path):
        print(f"The file {file_path} does not exist.")
    

In [12]:
# Clean up tmp files
for arxiv_number in arxiv_number_list:
    os.system(f"rm -rf {arxiv_number}/*.lua")
    os.system(f"rm -rf {arxiv_number}/*.aux")
    os.system(f"rm -rf {arxiv_number}/*.log")
    os.system(f"rm -rf {arxiv_number}/*.fls")
    os.system(f"rm -rf {arxiv_number}/*.fdb_latexmk")
    os.system(f"rm -rf {arxiv_number}/_markdown_{arxiv_number}_execution")

In [18]:
Clean up the output PDF [Be cautious!], comment out to run, uncomment after running

for arxiv_number in arxiv_number_list[:1]:
    os.system(f"rm -rf {arxiv_number}/*.pdf")

SyntaxError: invalid syntax (2051586890.py, line 1)

Preprocessing:
1. replace ## -> #, use new name
2. replace "===" to ""
3. replace $ []$ to $[]$

--
manual:
overlong equation can be used with $$..$$